In [41]:
import numpy as np
%load_ext autoreload
%autoreload 2

import torch
import sys

sys.path.append('..')

from algorithms.sequence_models.config import TrainConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is ", device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
device is  cuda


In [42]:
from minigrid.wrappers import ImgObsWrapper
from algorithms.sequence_models.decision_S4.dts4 import DecisionS4
from data.door_key_dataset import DoorKeyDataset
from comparison_experiment import AutomatedComparisonExperiment
import gymnasium as gym


inner_env = gym.make('MiniGrid-DoorKey-5x5-v0')
inner_env_human = gym.make('MiniGrid-DoorKey-5x5-v0', render_mode='human')
env = ImgObsWrapper(inner_env)
env_human = ImgObsWrapper(inner_env_human)


max_eval_ep_len = env.max_steps # todo change for other envs
context_len = max_eval_ep_len + 2  # todo change for other envs (DECREASE TO IMPROVE SPEED)

config = TrainConfig(max_eval_ep_len=max_eval_ep_len, context_len=max_eval_ep_len)

n_trajectories = 20_000 # todo is this enough / too much?
experiment_name = f"experiment_door_key_dataset_size={n_trajectories}"
if DoorKeyDataset.exists(experiment_name):
    traj_dataset = DoorKeyDataset.load(experiment_name)
else:
    reward_scale = 1 # todo change for other envs
    traj_dataset = DoorKeyDataset(env, n_trajectories=n_trajectories, reward_scale=reward_scale, prob_keeping_nonzero=0.01)
    traj_dataset.save(experiment_name)

comparison_experiment = AutomatedComparisonExperiment(
    env=env,
    traj_dataset=traj_dataset,
    config=config,
    device=device,
    rtgs_for_train_eval=[0.3, 0.5, 0.7, 1], # todo change for other envs
    rtgs_final_test=np.linspace(0, 1.1, 10, endpoint=True), # todo change for other envs
    env_name=f"MiniGrid-DoorKey-5x5-v0", # todo change for other envs
    experiment_name="automated_check_after_debug"
)

/home/shayan/miniconda3/envs/rl-explore/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.max_steps to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.max_steps` for environment variables or `env.get_wrapper_attr('max_steps')` that will search the reminding wrappers.
  logger.warn(


s4 : number of parameters 630811
dt : number of parameters 1159579


In [43]:
epoch = 100

In [ ]:
comparison_experiment.run_and_report(comparison_experiment.s4_experiment, epoch)

distribution of rewards in the dataset
training started


  0%|          | 0/100 [00:00<?, ?it/s]

time elapsed: 0:00:04
num of updates: 100
loss: 0.18059
lr 0.00001
grad_norm 0.15679


In [ ]:
comparison_experiment.run_and_report(comparison_experiment.dt_experiment, epoch)

In [ ]:
# see and record
from algorithms.evaluate_policy import evaluate_policy

policy = comparison_experiment.make_dt_policy(rtg=1)
evaluate_policy(policy, env_human, num_eval_ep=20)

In [ ]:
# see and record
from algorithms.evaluate_policy import evaluate_policy

policy = comparison_experiment.make_s4_policy(rtg=1)
evaluate_policy(policy, env_human, num_eval_ep=1)